In [2]:
import pandas as pd, numpy as np

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', None)

### ClinVar

In [38]:
clinvar_allpreds = pd.read_csv('~/Desktop/analysis files/cleaned_ClinVar_ALL_preds.csv', sep=',')

/var/folders/cc/jxs8yxmn00bb_43d9z_0f13c0000gn/T/ipykernel_48435/2626421186.py:1: DtypeWarning: Columns (72) have mixed types. Specify dtype option on import or set low_memory=False.
  clinvar_allpreds = pd.read_csv('~/Desktop/analysis files/cleaned_ClinVar_ALL_preds.csv', sep=',')


In [39]:
clinvar_allpreds.shape

(46050, 74)

In [40]:
clinvar_allpreds.head(3)

,#AlleleID,Type,Name,GeneID,GeneSymbol,HGNC_ID,ClinicalSignificance,ClinSigSimple,RS# (dbSNP),nsv/esv (dbVar),RCVaccession,PhenotypeIDS,PhenotypeList,Origin,OriginSimple,Assembly,ChromosomeAccession,Chromosome,Start,Stop,Cytogenetic,ReviewStatus,NumberSubmitters,OtherIDs,SubmitterCategories,VariationID,PositionVCF,ReferenceAlleleVCF,AlternateAlleleVCF,Variant (3-letter),Variant,LastEvaluated (Year),Uploaded_variation,Location,Allele,Gene,Feature,Feature_type,Consequence,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,Existing_variation,Extra,PositionVCF_dashed,gnomADe_AF,gnomADg_AF,gnomAD_AF,HGVSp,BinaryClinicalSignificance,SIFT_label,SIFT_score,PolyPhen_label,PolyPhen_score,BayesDel_label,BayesDel_score,CADD_PHRED_score,CADD_RAW_score,ClinPred_score,VEST4_score,VEST4_rankscore,EVE_label,EVE_score,REVEL_score,PrimateAI_score,AM_label,AM_score,UniprotID,Envision_predictions,QAFI_predictions,MetaRNN_score,MetaRNN_pred
0,317931,single nucleotide variant,NM_015665.6(AAAS):c.1597G>A (p.Gly533Arg),8086,AAAS,HGNC:13666,Benign/Likely benign,0,34451260,-,RCV000343022|RCV000886859,"MONDO:MONDO:0009279,MedGen:C0271742,OMIM:231550,Orphanet:869|MedGen:C3661900",Glucocorticoid deficiency with achalasia|not provided,germline,germline,GRCh38,NC_000012.12,12,53307533,53307533,12q13.13,"criteria provided, multiple submitters, no conflicts",3,ClinGen:CA6598796,2,309718,53307533,C,T,Gly533Arg,G533R,2024,3.445126e+07,12:53307533,T,ENSG00000094914,ENST00000209873.9,Transcript,missense_variant,1742,1597,533,G/R,Ggg/Agg,rs34451260,"IMPACT=MODERATE;STRAND=-1;SYMBOL=AAAS;SYMBOL_SOURCE=HGNC;HGNC_ID=HGNC:13666;CCDS=CCDS8856.1;ENSP=ENSP00000209873;SWISSPROT=Q9NRG9.206;UNIPARC=UPI0000039E40;UNIPROT_ISOFORM=Q9NRG9-1;HGVSc=ENST00000209873.9:c.1597G>A;HGVSp=ENSP00000209873.4:p.Gly533Arg;AF=0.0124;AFR_AF=0.0439;AMR_AF=0.0043;EAS_AF=0;EUR_AF=0.001;SAS_AF=0;gnomADe_AF=0.0007928;gnomADe_AFR_AF=0.02712;gnomADe_AMR_AF=0.001879;gnomADe_ASJ_AF=3.827e-05;gnomADe_EAS_AF=0;gnomADe_FIN_AF=0;gnomADe_MID_AF=0.001214;gnomADe_NFE_AF=2.428e-05;gnomADe_REMAINING_AF=0.002053;gnomADe_SAS_AF=9.275e-05;gnomADg_AF=0.007867;gnomADg_AFR_AF=0.02713;gnomADg_AMI_AF=0;gnomADg_AMR_AF=0.003529;gnomADg_ASJ_AF=0;gnomADg_EAS_AF=0;gnomADg_FIN_AF=0;gnomADg_MID_AF=0;gnomADg_NFE_AF=0.0001029;gnomADg_REMAINING_AF=0.004748;gnomADg_SAS_AF=0;MAX_AF=0.0439;MAX_AF_POPS=AFR;CLIN_SIG=benign,likely_benign;PHENO=1",53307533,0.000793,0.007867,0.000793,ENSP00000209873.4:p.Gly533Arg,B,deleterious_low_confidence,0.00,unknown,0.000,T,-0.362112,20.9,2.951810,0.062144,0.420,0.45743,NaN,NaN,0.257,0.526710,likely_benign,0.1192,Q9NRG9,0.977286,0.852,NaN,NaN
1,20084,single nucleotide variant,NM_015665.6(AAAS):c.787T>C (p.Ser263Pro),8086,AAAS,HGNC:13666,Pathogenic/Likely pathogenic,1,121918550,-,RCV000005348|RCV000311283|RCV000415076|RCV000624696,"MONDO:MONDO:0009279,MedGen:C0271742,OMIM:231550,Orphanet:869|MedGen:C3661900|Human Phenotype Ontology:HP:0002313,Human Phenotype Ontology:HP:0007191,MedGen:C0037771;Human Phenotype Ontology:HP:0001282,Human Phenotype Ontology:HP:0001347,Human Phenotype Ontology:HP:0006820,Human Phenotype Ontology:HP:0007184,Human Phenotype Ontology:HP:0007318,MONDO:MONDO:0007774,MedGen:C0151889,OMIM:145290;Human Phenotype Ontology:HP:0001352,Human Phenotype Ontology:HP:0003487,MedGen:C0034935|MeSH:D030342,MedGen:C0950123",Glucocorticoid deficiency with achalasia|not provided|Spastic paraparesis;Hyperreflexia;Babinski sign|Inborn genetic diseases,germline;unknown,germline,GRCh38,NC_000012.12,12,53309624,53309624,12q13.13,"criteria provided, multiple submitters, no conflicts",11,"ClinGen:CA117228,UniProtKB:Q9NRG9#VAR_012806,OMIM:605378.0007",3,5045,53309624,A,G,Ser263Pro,S263P,2024,1.219186e+08,12:53309624,G,ENSG00000094914,ENST00000209873.9,Transcript,missense_variant,932,787,263,S/P,Tca/Cca,"rs121918550,CM010150","IMPACT=MODERATE;STRAND=-1;SYMBOL=AAAS;SYMBOL_SOURCE=HGNC;HGNC_ID=HGNC:13666;CCDS=CCDS8856.1;ENSP=ENSP00000209873;SWISSPROT=Q9NRG9.206;UNIPARC=UPI0000039E40;UNIPROT_I

In [41]:
clinvar_allpreds[['BayesDel_label', 'BayesDel_score', 'CADD_PHRED_score', 'CADD_RAW_score', 'VEST4_score', 'EVE_label', 'EVE_score', 'REVEL_score', 'AM_label', 'AM_score', 'Envision_predictions', 'QAFI_predictions', 'MetaRNN_score', 'MetaRNN_pred']].isna().sum()

BayesDel_label             55
BayesDel_score             55
CADD_PHRED_score            0
CADD_RAW_score              0
VEST4_score                 0
EVE_label               15760
EVE_score               15760
REVEL_score              3876
AM_label                 2879
AM_score                 2879
Envision_predictions     3337
QAFI_predictions        16427
MetaRNN_score           36563
MetaRNN_pred            36563
dtype: int64

In [42]:
for col in clinvar_allpreds.columns:
    if col.endswith('label') or col.endswith('pred'):
        print(f"{col}: {clinvar_allpreds[col].unique()}")

SIFT_label: ['deleterious_low_confidence' 'deleterious' 'tolerated'
 'tolerated_low_confidence' nan]
PolyPhen_label: ['unknown' 'probably_damaging' 'benign' 'possibly_damaging' nan]
BayesDel_label: ['T' 'D' nan]
EVE_label: [nan 'Pathogenic' 'Benign' 'Uncertain']
AM_label: ['likely_benign' 'likely_pathogenic' 'ambiguous' nan]
MetaRNN_pred: [nan 'D' 'T' '.']


In [43]:
mappings = {
    'SIFT_label': {'deleterious': 1, 'deleterious_low_confidence': 1, 'tolerated': 0, 'tolerated_low_confidence': 0},
    'PolyPhen_label': {'probably_damaging': 1, 'possibly_damaging': 1, 'benign': 0},
    'BayesDel_label': {'T': 0, 'D': 1},
    'EVE_label': {'Benign': 0, 'Pathogenic': 1},
    'AM_label': {'likely_benign': 0, 'likely_pathogenic': 1},
    'MetaRNN_pred': {'T': 0, 'D': 1}
}

for col, mapping in mappings.items():
    clinvar_allpreds[col] = clinvar_allpreds[col].map(mapping).astype('Int64')

In [54]:
clinvar_allpreds.head(3)

,#AlleleID,Type,Name,GeneID,GeneSymbol,HGNC_ID,ClinicalSignificance,ClinSigSimple,RS# (dbSNP),nsv/esv (dbVar),RCVaccession,PhenotypeIDS,PhenotypeList,Origin,OriginSimple,Assembly,ChromosomeAccession,Chromosome,Start,Stop,Cytogenetic,ReviewStatus,NumberSubmitters,OtherIDs,SubmitterCategories,VariationID,PositionVCF,ReferenceAlleleVCF,AlternateAlleleVCF,Variant (3-letter),Variant,LastEvaluated (Year),Uploaded_variation,Location,Allele,Gene,Feature,Feature_type,Consequence,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,Existing_variation,Extra,PositionVCF_dashed,gnomADe_AF,gnomADg_AF,gnomAD_AF,HGVSp,BinaryClinicalSignificance,SIFT_label,SIFT_score,PolyPhen_label,PolyPhen_score,BayesDel_label,BayesDel_score,CADD_PHRED_score,CADD_RAW_score,ClinPred_score,VEST4_score,VEST4_rankscore,EVE_label,EVE_score,REVEL_score,PrimateAI_score,AM_label,AM_score,UniprotID,Envision_predictions,QAFI_predictions,MetaRNN_score,MetaRNN_pred
0,317931,single nucleotide variant,NM_015665.6(AAAS):c.1597G>A (p.Gly533Arg),8086,AAAS,HGNC:13666,Benign/Likely benign,0,34451260,-,RCV000343022|RCV000886859,"MONDO:MONDO:0009279,MedGen:C0271742,OMIM:231550,Orphanet:869|MedGen:C3661900",Glucocorticoid deficiency with achalasia|not provided,germline,germline,GRCh38,NC_000012.12,12,53307533,53307533,12q13.13,"criteria provided, multiple submitters, no conflicts",3,ClinGen:CA6598796,2,309718,53307533,C,T,Gly533Arg,G533R,2024,3.445126e+07,12:53307533,T,ENSG00000094914,ENST00000209873.9,Transcript,missense_variant,1742,1597,533,G/R,Ggg/Agg,rs34451260,"IMPACT=MODERATE;STRAND=-1;SYMBOL=AAAS;SYMBOL_SOURCE=HGNC;HGNC_ID=HGNC:13666;CCDS=CCDS8856.1;ENSP=ENSP00000209873;SWISSPROT=Q9NRG9.206;UNIPARC=UPI0000039E40;UNIPROT_ISOFORM=Q9NRG9-1;HGVSc=ENST00000209873.9:c.1597G>A;HGVSp=ENSP00000209873.4:p.Gly533Arg;AF=0.0124;AFR_AF=0.0439;AMR_AF=0.0043;EAS_AF=0;EUR_AF=0.001;SAS_AF=0;gnomADe_AF=0.0007928;gnomADe_AFR_AF=0.02712;gnomADe_AMR_AF=0.001879;gnomADe_ASJ_AF=3.827e-05;gnomADe_EAS_AF=0;gnomADe_FIN_AF=0;gnomADe_MID_AF=0.001214;gnomADe_NFE_AF=2.428e-05;gnomADe_REMAINING_AF=0.002053;gnomADe_SAS_AF=9.275e-05;gnomADg_AF=0.007867;gnomADg_AFR_AF=0.02713;gnomADg_AMI_AF=0;gnomADg_AMR_AF=0.003529;gnomADg_ASJ_AF=0;gnomADg_EAS_AF=0;gnomADg_FIN_AF=0;gnomADg_MID_AF=0;gnomADg_NFE_AF=0.0001029;gnomADg_REMAINING_AF=0.004748;gnomADg_SAS_AF=0;MAX_AF=0.0439;MAX_AF_POPS=AFR;CLIN_SIG=benign,likely_benign;PHENO=1",53307533,0.000793,0.007867,0.000793,ENSP00000209873.4:p.Gly533Arg,B,1,0.00,<NA>,0.000,0,-0.362112,20.9,2.951810,0.062144,0.420,0.45743,<NA>,NaN,0.257,0.526710,0,0.1192,Q9NRG9,0.977286,0.852,NaN,<NA>
1,20084,single nucleotide variant,NM_015665.6(AAAS):c.787T>C (p.Ser263Pro),8086,AAAS,HGNC:13666,Pathogenic/Likely pathogenic,1,121918550,-,RCV000005348|RCV000311283|RCV000415076|RCV000624696,"MONDO:MONDO:0009279,MedGen:C0271742,OMIM:231550,Orphanet:869|MedGen:C3661900|Human Phenotype Ontology:HP:0002313,Human Phenotype Ontology:HP:0007191,MedGen:C0037771;Human Phenotype Ontology:HP:0001282,Human Phenotype Ontology:HP:0001347,Human Phenotype Ontology:HP:0006820,Human Phenotype Ontology:HP:0007184,Human Phenotype Ontology:HP:0007318,MONDO:MONDO:0007774,MedGen:C0151889,OMIM:145290;Human Phenotype Ontology:HP:0001352,Human Phenotype Ontology:HP:0003487,MedGen:C0034935|MeSH:D030342,MedGen:C0950123",Glucocorticoid deficiency with achalasia|not provided|Spastic paraparesis;Hyperreflexia;Babinski sign|Inborn genetic diseases,germline;unknown,germline,GRCh38,NC_000012.12,12,53309624,53309624,12q13.13,"criteria provided, multiple submitters, no conflicts",11,"ClinGen:CA117228,UniProtKB:Q9NRG9#VAR_012806,OMIM:605378.0007",3,5045,53309624,A,G,Ser263Pro,S263P,2024,1.219186e+08,12:53309624,G,ENSG00000094914,ENST00000209873.9,Transcript,missense_variant,932,787,263,S/P,Tca/Cca,"rs121918550,CM010150","IMPACT=MODERATE;STRAND=-1;SYMBOL=AAAS;SYMBOL_SOURCE=HGNC;HGNC_ID=HGNC:13666;CCDS=CCDS8856.1;ENSP=ENSP00000209873;SWISSPROT=Q9NRG9.206;UNIPARC=UPI0000039E40;UNIPROT_ISOFORM=Q9NRG9-1;HGVSc=ENST00000209873.

### Humsavar

In [135]:
humsavar_allpreds = pd.read_csv('~/Desktop/analysis files/cleaned_Humsavar_ALL_preds.csv', sep=',')

In [126]:
humsavar_allpreds.shape

(1108, 71)

In [127]:
humsavar_allpreds.head(3)

,Main gene name,Uniprot,FTId,AA change,Variant category,dbSNP,Disease name,Source,Variant,BinaryClinicalSignificance,MANE-Select ID,Ensembl ID,Protein ID,RefSeq Nucleotide ID,RefSeq Protein ID,Error,#Uploaded_variation,Location,Allele,Consequence,IMPACT,GeneSymbol,Gene,Feature_type,Feature,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,Existing_variation,REF_ALLELE,HGNC_ID,ENSP,SWISSPROT,TREMBL,UNIPARC,UNIPROT_ISOFORM,SIFT_label,SIFT_score,PolyPhen_label,PolyPhen_score,AF,CLIN_SIG,SOMATIC,PHENO,PUBMED,TRANSCRIPTION_FACTORS,REVEL_score,BLOSUM62,EVE_label,EVE_score,CADD_PHRED_score,CADD_RAW_score,AM_label,AM_score,BayesDel_label,BayesDel_score,PrimateAI_label,PrimateAI_score,Uniprot_acc,Uniprot_entry,clinvar_id,clinvar_review,ClinPred_score,NaN_count,UniprotID,Envision_predictions,QAFI_predictions,MetaRNN_score,MetaRNN_pred
0,ABTB3,A6QL63,VAR_042534,p.Ala1002Asp,LB/B,rs11610050,NaN,2025,A1002D,B,ENST00000280758.10,ENST00000280758,ENSP00000280758.5,NM_001018072.2,NP_001018082.1,NaN,rs11610050,12:107651687-107651687,A,missense_variant,MODERATE,ABTB3,ENSG00000151136,Transcript,ENST00000280758.10,3512,3005,1002,A/D,gCt/gAt,rs11610050,C,HGNC:23844,ENSP00000280758,A6QL63.129,-,UPI000051901E,A6QL63-1,deleterious_low_confidence,0.00,probably_damaging,0.972,-,-,-,-,-,-,0.649,-2,-,-,31.0,5.339176,likely_pathogenic,0.9994,D,0.356338,D,0.848108232021,"A6QL63,A6QL63-2,A6QL63-5,A6QL63-4","BTBDB_HUMAN,BTBDB_HUMAN,BTBDB_HUMAN,BTBDB_HUMAN",-,-,0.999407649040222,0.0,A6QL63,0.772274,NaN,NaN,NaN
1,ABTB3,A6QL63,VAR_042535,p.Gly1076Ser,LB/B,rs12303478,NaN,2025,G1076S,B,ENST00000280758.10,ENST00000280758,ENSP00000280758.5,NM_001018072.2,NP_001018082.1,NaN,rs12303478,12:107657629-107657629,A,missense_variant,MODERATE,ABTB3,ENSG00000151136,Transcript,ENST00000280758.10,3733,3226,1076,G/S,Ggc/Agc,"rs12303478,COSV104383452",G,HGNC:23844,ENSP00000280758,A6QL63.129,-,UPI000051901E,A6QL63-1,tolerated_low_confidence,0.09,benign,0.049,0.1048,-,"0,1","0,1",-,-,0.072,0,-,-,22.4,3.396815,likely_benign,0.0921,T,-0.583583,T,0.613067984581,"A6QL63,A6QL63-2,A6QL63-5,A6QL63-4","BTBDB_HUMAN,BTBDB_HUMAN,BTBDB_HUMAN,BTBDB_HUMAN",-,-,0.0157182390193226,0.0,A6QL63,0.978557,NaN,NaN,NaN
2,ABTB3,A6QL63,VAR_055560,p.Gly448Ser,LB/B,rs1558781,NaN,2025,G448S,B,ENST00000280758.10,ENST00000280758,ENSP00000280758.5,NM_001018072.2,NP_001018082.1,NaN,rs1558781,12:107543991-107543991,A,missense_variant,MODERATE,ABTB3,ENSG00000151136,Transcript,ENST00000280758.10,1849,1342,448,G/S,Ggc/Agc,"rs1558781,COSV99774826",G,HGNC:23844,ENSP00000280758,A6QL63.129,-,UPI000051901E,A6QL63-1,tolerated_low_confidence,0.43,benign,0.022,0.1763,-,"0,1","0,1",-,-,0.102,0,-,-,22.3,3.373019,likely_benign,0.0774,T,-0.626984,T,0.485189527273,"A6QL63,A6QL63-2,A6QL63-3,H0YHR1,C9JV03","BTBDB_HUMAN,BTBDB_HUMAN,BTBDB_HUMAN,H0YHR1_HUMAN,C9JV03_HUMAN",-,-,0.0154826749531858,0.0,A6QL63,0.983750,NaN,NaN,NaN


In [128]:
humsavar_allpreds[['BayesDel_label', 'BayesDel_score', 'CADD_PHRED_score', 'CADD_RAW_score', 'EVE_label', 'EVE_score', 'REVEL_score', 'AM_label', 'AM_score', 'Envision_predictions', 'QAFI_predictions', 'MetaRNN_score', 'MetaRNN_pred']].isna().sum()

BayesDel_label            0
BayesDel_score            0
CADD_PHRED_score          0
CADD_RAW_score            0
EVE_label                 0
EVE_score                 0
REVEL_score               0
AM_label                  0
AM_score                  0
Envision_predictions    246
QAFI_predictions        834
MetaRNN_score           716
MetaRNN_pred            716
dtype: int64

In [136]:
for col in humsavar_allpreds.columns:
    if col.endswith('label') or col.endswith('pred'):
        print(f"{col}: {humsavar_allpreds[col].unique()}")

SIFT_label: ['deleterious_low_confidence' 'tolerated_low_confidence' 'deleterious'
 'tolerated' '-']
PolyPhen_label: ['probably_damaging' 'benign' 'possibly_damaging' 'unknown' '-']
EVE_label: ['-' 'Pathogenic' 'Benign' 'Uncertain']
AM_label: ['likely_pathogenic' 'likely_benign' 'ambiguous' '-']
BayesDel_label: ['D' 'T' '-']
PrimateAI_label: ['D' 'T' '-']
MetaRNN_pred: [nan 'D' 'T' '.']


In [137]:
mappings = {
    'SIFT_label': {'deleterious': 1, 'deleterious_low_confidence': 1, 'tolerated': 0, 'tolerated_low_confidence': 0},
    'PolyPhen_label': {'probably_damaging': 1, 'possibly_damaging': 1, 'benign': 0},
    'BayesDel_label': {'T': 0, 'D': 1},
    'EVE_label': {'Benign': 0, 'Pathogenic': 1},
    'AM_label': {'likely_benign': 0, 'likely_pathogenic': 1},
    'MetaRNN_pred': {'T': 0, 'D': 1}
}

for col, mapping in mappings.items():
    humsavar_allpreds[col] = humsavar_allpreds[col].map(mapping).astype('Int64')

In [138]:
humsavar_allpreds.head(3)

,Main gene name,Uniprot,FTId,AA change,Variant category,dbSNP,Disease name,Source,Variant,BinaryClinicalSignificance,MANE-Select ID,Ensembl ID,Protein ID,RefSeq Nucleotide ID,RefSeq Protein ID,Error,#Uploaded_variation,Location,Allele,Consequence,IMPACT,GeneSymbol,Gene,Feature_type,Feature,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,Existing_variation,REF_ALLELE,HGNC_ID,ENSP,SWISSPROT,TREMBL,UNIPARC,UNIPROT_ISOFORM,SIFT_label,SIFT_score,PolyPhen_label,PolyPhen_score,AF,CLIN_SIG,SOMATIC,PHENO,PUBMED,TRANSCRIPTION_FACTORS,REVEL_score,BLOSUM62,EVE_label,EVE_score,CADD_PHRED_score,CADD_RAW_score,AM_label,AM_score,BayesDel_label,BayesDel_score,PrimateAI_label,PrimateAI_score,Uniprot_acc,Uniprot_entry,clinvar_id,clinvar_review,ClinPred_score,NaN_count,UniprotID,Envision_predictions,QAFI_predictions,MetaRNN_score,MetaRNN_pred
0,ABTB3,A6QL63,VAR_042534,p.Ala1002Asp,LB/B,rs11610050,NaN,2025,A1002D,B,ENST00000280758.10,ENST00000280758,ENSP00000280758.5,NM_001018072.2,NP_001018082.1,NaN,rs11610050,12:107651687-107651687,A,missense_variant,MODERATE,ABTB3,ENSG00000151136,Transcript,ENST00000280758.10,3512,3005,1002,A/D,gCt/gAt,rs11610050,C,HGNC:23844,ENSP00000280758,A6QL63.129,-,UPI000051901E,A6QL63-1,1,0.00,1,0.972,-,-,-,-,-,-,0.649,-2,<NA>,-,31.0,5.339176,1,0.9994,1,0.356338,D,0.848108232021,"A6QL63,A6QL63-2,A6QL63-5,A6QL63-4","BTBDB_HUMAN,BTBDB_HUMAN,BTBDB_HUMAN,BTBDB_HUMAN",-,-,0.999407649040222,0.0,A6QL63,0.772274,NaN,NaN,<NA>
1,ABTB3,A6QL63,VAR_042535,p.Gly1076Ser,LB/B,rs12303478,NaN,2025,G1076S,B,ENST00000280758.10,ENST00000280758,ENSP00000280758.5,NM_001018072.2,NP_001018082.1,NaN,rs12303478,12:107657629-107657629,A,missense_variant,MODERATE,ABTB3,ENSG00000151136,Transcript,ENST00000280758.10,3733,3226,1076,G/S,Ggc/Agc,"rs12303478,COSV104383452",G,HGNC:23844,ENSP00000280758,A6QL63.129,-,UPI000051901E,A6QL63-1,0,0.09,0,0.049,0.1048,-,"0,1","0,1",-,-,0.072,0,<NA>,-,22.4,3.396815,0,0.0921,0,-0.583583,T,0.613067984581,"A6QL63,A6QL63-2,A6QL63-5,A6QL63-4","BTBDB_HUMAN,BTBDB_HUMAN,BTBDB_HUMAN,BTBDB_HUMAN",-,-,0.0157182390193226,0.0,A6QL63,0.978557,NaN,NaN,<NA>
2,ABTB3,A6QL63,VAR_055560,p.Gly448Ser,LB/B,rs1558781,NaN,2025,G448S,B,ENST00000280758.10,ENST00000280758,ENSP00000280758.5,NM_001018072.2,NP_001018082.1,NaN,rs1558781,12:107543991-107543991,A,missense_variant,MODERATE,ABTB3,ENSG00000151136,Transcript,ENST00000280758.10,1849,1342,448,G/S,Ggc/Agc,"rs1558781,COSV99774826",G,HGNC:23844,ENSP00000280758,A6QL63.129,-,UPI000051901E,A6QL63-1,0,0.43,0,0.022,0.1763,-,"0,1","0,1",-,-,0.102,0,<NA>,-,22.3,3.373019,0,0.0774,0,-0.626984,T,0.485189527273,"A6QL63,A6QL63-2,A6QL63-3,H0YHR1,C9JV03","BTBDB_HUMAN,BTBDB_HUMAN,BTBDB_HUMAN,H0YHR1_HUMAN,C9JV03_HUMAN",-,-,0.0154826749531858,0.0,A6QL63,0.983750,NaN,NaN,<NA>


### Simplify ClinHum data to single tidy dataset

In [139]:
humsavar_allpreds["Variant category"].unique()

array(['LB/B', 'LP/P'], dtype=object)

In [140]:
# columns in ClinVar order
cols_to_keep = [
    'GeneSymbol', 'ClinSigSimple', 'Variant', 'UniprotID',
    'SIFT_label', 'SIFT_score', 'PolyPhen_label', 'PolyPhen_score',
    'BayesDel_label', 'BayesDel_score', 'CADD_PHRED_score', 'CADD_RAW_score',
    'ClinPred_score', 'VEST4_score', 'EVE_label', 'EVE_score',
    'REVEL_score', 'PrimateAI_score', 'AM_label', 'AM_score',
    'Envision_predictions', 'QAFI_predictions', 'MetaRNN_score', 'MetaRNN_pred'
]

clinvar_subset = clinvar_allpreds[cols_to_keep]

test = humsavar_allpreds.rename(columns={'Variant category': 'ClinSigSimple',})
test['ClinSigSimple'] = test['ClinSigSimple'].map({'LB/B': 0, 'LP/P': 1})
test['VEST4_score'] = np.nan

humsavar_subset = test[cols_to_keep]

combined = pd.concat([clinvar_subset, humsavar_subset], ignore_index=True)

In [141]:
combined.head()

,GeneSymbol,ClinSigSimple,Variant,UniprotID,SIFT_label,SIFT_score,PolyPhen_label,PolyPhen_score,BayesDel_label,BayesDel_score,CADD_PHRED_score,CADD_RAW_score,ClinPred_score,VEST4_score,EVE_label,EVE_score,REVEL_score,PrimateAI_score,AM_label,AM_score,Envision_predictions,QAFI_predictions,MetaRNN_score,MetaRNN_pred
0,AAAS,0,G533R,Q9NRG9,1,0.00,<NA>,0.000,0,-0.362112,20.9,2.951810,0.062144,0.420,<NA>,NaN,0.257,0.52671,0,0.1192,0.977286,0.852,NaN,<NA>
1,AAAS,1,S263P,Q9NRG9,1,0.00,1,0.992,1,0.25831,29.2,5.211912,0.980945,0.983,1,0.951397,0.844,0.71898,1,0.9852,0.860283,0.458,NaN,<NA>
2,AAAS,1,A167V,Q9NRG9,1,0.03,1,0.969,1,0.571933,34.0,5.972388,0.998261,0.980,1,0.853707,0.92,0.760144,1,0.8614,0.918544,0.766,NaN,<NA>
3,AAAS,1,H160R,Q9NRG9,1,0.00,1,0.952,1,0.553083,27.5,4.924293,0.99862,0.985,1,0.963709,0.919,0.675616,1,0.9973,0.828964,0.536,NaN,<NA>
4,AAAS,1,R155H,Q9NRG9,1,0.02,1,0.980,1,0.449379,29.4,5.229742,0.99574,0.898,1,0.717298,0.909,0.770769,1,0.6977,0.897079,0.647,NaN,<NA>


In [121]:
combined.isna().sum()

GeneSymbol                  0
ClinSigSimple               0
Variant                     0
UniprotID                   0
SIFT_label               1099
SIFT_score               1099
PolyPhen_label           6883
PolyPhen_score           3322
BayesDel_label             69
BayesDel_score             55
CADD_PHRED_score            0
CADD_RAW_score              0
ClinPred_score            141
VEST4_score              1108
EVE_label               22852
EVE_score               15760
REVEL_score              3876
PrimateAI_score           510
AM_label                 6953
AM_score                 2879
Envision_predictions     3583
QAFI_predictions        17261
MetaRNN_score           37279
MetaRNN_pred            37388
dtype: int64

In [143]:
combined.shape

(47158, 24)

In [144]:
missing_values = ['<NA>', 'NA', 'NaN', 'nan', 'None', '', 'null']
for col in combined.columns:
    combined[col] = combined[col].astype(str).replace(missing_values, np.nan)

In [131]:
combined.head()

,GeneSymbol,ClinSigSimple,Variant,UniprotID,SIFT_label,SIFT_score,PolyPhen_label,PolyPhen_score,BayesDel_label,BayesDel_score,CADD_PHRED_score,CADD_RAW_score,ClinPred_score,VEST4_score,EVE_label,EVE_score,REVEL_score,PrimateAI_score,AM_label,AM_score,Envision_predictions,QAFI_predictions,MetaRNN_score,MetaRNN_pred
0,AAAS,0,G533R,Q9NRG9,1.0,0.00,NaN,0.000,0.0,-0.362112,20.9,2.951810,0.062144,0.420,NaN,NaN,0.257,0.52671,0.0,0.1192,0.977286,0.852,NaN,NaN
1,AAAS,1,S263P,Q9NRG9,1.0,0.00,1.0,0.992,1.0,0.25831,29.2,5.211912,0.980945,0.983,1.0,0.951397,0.844,0.71898,1.0,0.9852,0.860283,0.458,NaN,NaN
2,AAAS,1,A167V,Q9NRG9,1.0,0.03,1.0,0.969,1.0,0.571933,34.0,5.972388,0.998261,0.980,1.0,0.853707,0.92,0.760144,1.0,0.8614,0.918544,0.766,NaN,NaN
3,AAAS,1,H160R,Q9NRG9,1.0,0.00,1.0,0.952,1.0,0.553083,27.5,4.924293,0.99862,0.985,1.0,0.963709,0.919,0.675616,1.0,0.9973,0.828964,0.536,NaN,NaN
4,AAAS,1,R155H,Q9NRG9,1.0,0.02,1.0,0.980,1.0,0.449379,29.4,5.229742,0.99574,0.898,1.0,0.717298,0.909,0.770769,1.0,0.6977,0.897079,0.647,NaN,NaN


In [122]:
# load extra QAFI scores, align column names
new = (pd.read_csv("clinvarall.csv")
         .rename(columns={'uniprot': 'UniprotID',
                          'variant': 'Variant',
                          'qafi1': 'QAFI_predictions'}))

# QAFI convention: ↑ benign, ↓ pathogenic  →  multiply by –1 so ↑ pathogenic
new['QAFI_predictions'] = -new['QAFI_predictions']

# merge and fill gaps only
combined = combined.merge(new[['UniprotID', 'Variant', 'QAFI_predictions']],
                          on=['UniprotID', 'Variant'],
                          how='left',
                          suffixes=('', '_new'))

combined['QAFI_predictions'] = combined['QAFI_predictions'].fillna(
                                   combined['QAFI_predictions_new'])
combined.drop(columns='QAFI_predictions_new', inplace=True)

In [140]:
combined.isna().sum()

GeneSymbol                  0
ClinSigSimple               0
Variant                     0
UniprotID                   0
SIFT_label               1099
SIFT_score               1099
PolyPhen_label           6891
PolyPhen_score           3330
BayesDel_label             69
BayesDel_score             55
CADD_PHRED_score            0
CADD_RAW_score              0
ClinPred_score            141
VEST4_score              1108
EVE_label               22860
EVE_score               15768
REVEL_score              3876
PrimateAI_score           510
AM_label                 6958
AM_score                 2883
Envision_predictions     3583
QAFI_predictions         4687
MetaRNN_score           37287
MetaRNN_pred            37396
dtype: int64

In [135]:
combined['VEST4_score'].agg(['min', 'max'])

min    0.0
max    1.0
Name: VEST4_score, dtype: float64

Label 0 has higher (more damaging) scores: 0 = pathogenic, 1 = benign; in **QAFI** and **Envision**.

Thus, we must flip the labels and rerun.

In [132]:
combined['QAFI_predictions'].agg(['min', 'max'])

min    0.1540
max    1.5025
Name: QAFI_predictions, dtype: float64

In [133]:
combined['Envision_predictions'].agg(['min', 'max'])

min    0.421637
max    1.127760
Name: Envision_predictions, dtype: float64

In [ ]:
combined['QAFI_predictions']   *= -1
combined['Envision_predictions'] *= -1

In [137]:
combined['QAFI_predictions'].agg(['min', 'max'])

min   -1.5025
max   -0.1540
Name: QAFI_predictions, dtype: float64

In [138]:
combined['Envision_predictions'].agg(['min', 'max'])

min   -1.127760
max   -0.421637
Name: Envision_predictions, dtype: float64

In [141]:
combined.to_csv('~/Desktop/analysis files/ClinHum_ALL_preds.csv', index=False)

### Table: Percentage of missing predictions for all tools for the combined Clinvar and Humsavar dataset

In [3]:
combined = pd.read_csv('~/Desktop/analysis files/ClinHum_ALL_preds_cleaned.csv', sep=',')

/var/folders/cc/jxs8yxmn00bb_43d9z_0f13c0000gn/T/ipykernel_19535/2962565520.py:1: DtypeWarning: Columns (12,17) have mixed types. Specify dtype option on import or set low_memory=False.
  combined = pd.read_csv('~/Desktop/analysis files/ClinHum_ALL_preds_cleaned.csv', sep=',')


In [4]:
combined.columns

Index(['GeneSymbol', 'ClinSigSimple', 'variant', 'uniprot', 'SIFT_label',
       'SIFT_score', 'PolyPhen_label', 'PolyPhen_score', 'BayesDel (binary)',
       'BayesDel', 'CADD', 'CADD (RAW)', 'ClinPred_score', 'VEST4',
       'EVE (binary)', 'EVE', 'REVEL', 'PrimateAI_score',
       'AlphaMissense (binary)', 'AlphaMissense', 'Envision', 'QAFI',
       'MetaRNN', 'MetaRNN (binary)'],
      dtype='object')

In [5]:
combined.head()

,GeneSymbol,ClinSigSimple,variant,uniprot,SIFT_label,SIFT_score,PolyPhen_label,PolyPhen_score,BayesDel (binary),BayesDel,CADD,CADD (RAW),ClinPred_score,VEST4,EVE (binary),EVE,REVEL,PrimateAI_score,AlphaMissense (binary),AlphaMissense,Envision,QAFI,MetaRNN,MetaRNN (binary)
0,AAAS,0,G533R,Q9NRG9,1.0,0.00,NaN,0.000,0.0,-0.362112,20.9,2.951810,0.062144,0.420,NaN,NaN,0.257,0.52671,0.0,0.1192,-0.977286,-0.852,NaN,NaN
1,AAAS,1,S263P,Q9NRG9,1.0,0.00,1.0,0.992,1.0,0.258310,29.2,5.211912,0.980945,0.983,1.0,0.951397,0.844,0.71898,1.0,0.9852,-0.860283,-0.458,NaN,NaN
2,AAAS,1,A167V,Q9NRG9,1.0,0.03,1.0,0.969,1.0,0.571933,34.0,5.972388,0.998261,0.980,1.0,0.853707,0.920,0.760144,1.0,0.8614,-0.918544,-0.766,NaN,NaN
3,AAAS,1,H160R,Q9NRG9,1.0,0.00,1.0,0.952,1.0,0.553083,27.5,4.924293,0.99862,0.985,1.0,0.963709,0.919,0.675616,1.0,0.9973,-0.828964,-0.536,NaN,NaN
4,AAAS,1,R155H,Q9NRG9,1.0,0.02,1.0,0.980,1.0,0.449379,29.4,5.229742,0.99574,0.898,1.0,0.717298,0.909,0.770769,1.0,0.6977,-0.897079,-0.647,NaN,NaN


In [6]:
combined.isna().sum()

GeneSymbol                    0
ClinSigSimple                 0
variant                       0
uniprot                       0
SIFT_label                 1099
SIFT_score                 1099
PolyPhen_label             6891
PolyPhen_score             3330
BayesDel (binary)            69
BayesDel                     69
CADD                          0
CADD (RAW)                    0
ClinPred_score              141
VEST4                      1108
EVE (binary)              22860
EVE                       16735
REVEL                      3975
PrimateAI_score             510
AlphaMissense (binary)     6958
AlphaMissense              3202
Envision                   3583
QAFI                       4687
MetaRNN                   37396
MetaRNN (binary)          37396
dtype: int64

In [7]:
score_cols  = ['CADD','MetaRNN','Envision','QAFI','EVE','AlphaMissense','BayesDel','REVEL','VEST4']

# % of variants with no preds for each predictor
missing_df = (
    combined[score_cols].isna().mean()
             .mul(100).round(1)
             .rename('ClinVar + Humsavar')
             .reset_index()
             .rename(columns={'index':'Predictor'})
)

In [8]:
missing_df

,Predictor,ClinVar + Humsavar
0,CADD,0.0
1,MetaRNN,79.3
2,Envision,7.6
3,QAFI,9.9
4,EVE,35.5
5,AlphaMissense,6.8
6,BayesDel,0.1
7,REVEL,8.4
8,VEST4,2.3


In [9]:
missing_df.to_latex("missing_preds_table_PreAnalysis.tex", index=False)